# Import library 

In [ ]:
import sys
sys.argv = ['jupyter']  # Clear the parameters passed to Jupyter
sys.path.append('../')

In [ ]:
# Import library
import sys,os,time,json,functools,argparse,heapq,pickle,ast
sys.path.append('../')
from my_packages import cheminfo_tools,functions_new,evaluation,peaktools
from rdkit.DataStructs import TanimotoSimilarity

import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
import seaborn as sns
from scipy.stats import gaussian_kde



import os,json,ujson,ms_entropy,matchms,json,spectral_entropy,sys,time
sys.path.append('../')
import pandas as pd
import numpy as np
import networkx as nx
import spectral_entropy as se
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import spectrum_utils.spectrum as sus

from spectral_entropy.spectral_entropy import calculate_entropy
from spectral_entropy import similarity
from matchms.importing import load_from_json, load_from_mgf,load_from_msp, load_from_mzml, load_from_mzxml, load_from_usi
from tqdm import tqdm, trange
from ms_entropy import FlashEntropySearch,FlashEntropySearchCore
from my_packages import functions_new,ms2tools_new,cheminfo_tools,evaluation
from rdkit import Chem
from collections import Counter
from sklearn.metrics import roc_curve,auc,confusion_matrix
from FPSim2.io import create_db_file
from FPSim2 import FPSim2Engine
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.decomposition import PCA

from matchms import Spectrum
from matchms.similarity import ModifiedCosine,NeutralLossesCosine,CosineGreedy

# Load library files
## EDB

In [ ]:
# Load GNPS library file
t = time.time() 

# GNPS_FILE = '../msdb/GNPSLIBRARY_250514/ALL_GNPS_NO_PROPOGATED.mgf'
GNPS_CLEAN_FILE = '../msdb/GNPSLIBRARY_250514/ALL_GNPS_cleaned.mgf'
# GNPS_DEMO_FILE = '../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-demo.mgf'

# GNPS_INFO = list(load_from_mgf(GNPS_FILE))
GNPS_CLAEN_INFO = list(load_from_mgf(GNPS_CLEAN_FILE))
print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
# Load library spectral file
t = time.time()

# ISMS1_CSV = '../msdb/isdbMS1.csv' # 'id', 'formula', 'exactmass', 'smiles', 'inchi', 'inchikey', 'm+h','m+nh4', 'm+na'
# ISMS1_DF = pd.read_csv(ISMS1_CSV,low_memory=False)
# EMS1_CSV = '../msdb/edbMS1.csv' # 'id', 'pepmass', 'smiles'
# EMS1_DF = pd.read_csv(EMS1_CSV)

# Flash search - Load
FS_E_LIBRARY = functions_new.json_load('../msdb/FS_edb_info.json')
e_search = FlashEntropySearch()
FS_E_LIBRARY = e_search.build_index(FS_E_LIBRARY)

print(f'Finished in {(time.time() - t) / 60:.2f} min')

## ISDB

In [ ]:
# ISDB energy 10,20,40
ISDB_INFO = functions_new.json_load('../msdb/isdb_info.json')

In [ ]:
# ISDB energy = 10
t = time.time()
FS_IS_LIBRARY_e0 = functions_new.json_load('../msdb/FS_isdb_e0.json')
is0_search = FlashEntropySearch()
FS_IS0_LIBRARY = is0_search.build_index(FS_IS_LIBRARY_e0)

# ISDB energy = 20
t = time.time()
FS_IS_LIBRARY_e1 = functions_new.json_load('../msdb/FS_isdb_e1.json')
is1_search = FlashEntropySearch()
FS_IS1_LIBRARY = is1_search.build_index(FS_IS_LIBRARY_e1)

# ISDB energy = 40
t = time.time()
FS_IS_LIBRARY_e2 = functions_new.json_load('../msdb/FS_isdb_e2.json')
is2_search = FlashEntropySearch()
FS_IS2_LIBRARY = is2_search.build_index(FS_IS_LIBRARY_e2)
print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
# Load data
# Selected CCMSids
CCMSIDs = np.load('../msdb/data/idlist/H_qtof_non-redundant_CCMSIDs.npy')

# Library files
GNPS_INFO_FILE = '../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json'
GNPS_INFO = functions_new.json_load(GNPS_INFO_FILE)

# Tanimoto matrix
TANIMOTO_MATRIX_FILE = f'../msdb/data/structural_matrix/tanimoto_similarity_matrix.npy'
TANIMOTO_MATRIX = np.load(TANIMOTO_MATRIX_FILE) # Tanimoto similarity matrices
IDXtoCCMSID = functions_new.json_load('../msdb/data/idlist/INDEXtoCCMS.json')
CCMSIDtoIDX = {value:key for key,value in IDXtoCCMSID.items() }

# ALGORITHMs
G1,G2,G3,G4 = topology.get_groups()
ALGORITHMs = G1 + G2 + G3 + G4 + ['peak_percentage'] 
ALGORITHM = [a for a in ALGORITHMs if 'modified_' in a][0] # ALGORITHMS

# Spectral matrices
SPEC_SIM_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy'
SPEC_SIM_MATRIX = np.load(SPEC_SIM_MATRIX_FILE)

if ALGORITHM == 'neutral_loss':
    N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_peak_matrix.npy'  # expect for modified_cosine and neutral_loss
elif ALGORITHM == 'modified_cosine':
    N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_peak_matrix.npy'
else:
    N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/cosine_peak_matrix.npy'
N_PEAK_MATRIX = np.load(N_PEAK_MATRIX_FILE)

# MS1 matchi matrix(1: match, 0: not match)
MS1_MATCH_MATRIX = np.load(f'../msdb/data/structural_matrix/MS1match_matrix.npy')

## Npeak and entropy supplement

In [ ]:
# Supplement the library with additional information (e.g., entropy, n_peaks, classes)
# Param: FS_E_LIBRARY,FS_IS0_LIBRARY,FS_IS1_LIBRARY,FS_IS2_LIBRARY,
# Return: LIBRARIES with supplemented information 
for SPECTRUM in tqdm(FS_E_LIBRARY, total = len(FS_E_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    


In [ ]:
for SPECTRUM in tqdm(FS_IS0_LIBRARY, total = len(FS_IS0_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    SPECTRUM['peaks'] = PEAKs.tolist()
    
for SPECTRUM in tqdm(FS_IS1_LIBRARY, total = len(FS_IS1_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    SPECTRUM['peaks'] = PEAKs.tolist()

for SPECTRUM in tqdm(FS_IS2_LIBRARY, total = len(FS_IS2_LIBRARY)):
    PEAKs = SPECTRUM['peaks']
    SPECTRUM['n_peak'] = len(PEAKs)
    SPECTRUM['entropy'] = calculate_entropy(PEAKs)
    SPECTRUM['peaks'] = PEAKs.tolist()

In [ ]:
with open('../msdb/FS_isdb_e0.json', "w") as f:
    json.dump(FS_IS0_LIBRARY, f)

with open('../msdb/FS_isdb_e1.json', "w") as f:
    json.dump(FS_IS1_LIBRARY, f)

with open('../msdb/FS_isdb_e2.json', "w") as f:
    json.dump(FS_IS2_LIBRARY, f)

## Hqtof lib

In [ ]:
# Load GNPS library
GNPS_SPECTRA = list(load_from_mgf('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY.mgf'))
GNPS_INFO = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json') 

In [ ]:
# 2571 
# 
Hqtof_CCMSIDs = list(np.load('../msdb/data/hqtof/idlist/H_qtof_non-redundant_CCMSIDs.npy'))

# FS_SPECTRA = []
# for SPECTRUM in tqdm(GNPS_SPECTRA,total = len(GNPS_SPECTRA)):
#     CCMSID = SPECTRUM.metadata['spectrum_id']
#     if CCMSID in Hqtof_CCMSIDs:
#         PM = SPECTRUM.metadata['precursor_mz']
#         IONMODE = SPECTRUM.metadata['ionmode']
#         try:
#             SMILE = SPECTRUM.metadata['smiles']
#         except:
#             SMILE = GNPS_INFO[CCMSID]['CANONSMILES']
            
#         CHARGE = SPECTRUM.metadata['charge']
#         SCLASS = GNPS_INFO[CCMSID]['np_classifier_superclass']
        
#         peaks = np.column_stack((SPECTRUM.mz,SPECTRUM.intensities))
#         # peaks = se.clean_spectrum(peaks, max_mz=PM + 1.6)
#         FS_SPECTRA.append({
#             "id": CCMSID,
#             "precursor_mz": PM,
#             "peaks": peaks.tolist(),
#             "smile": SMILE,
#             "charge": CHARGE,
#             "ion_mode":IONMODE,
#             'superclass':[SCLASS]
#             })

# # Save
# sorted_list = sorted(FS_SPECTRA, key=lambda x: x['precursor_mz'])
# FS_GNPS_LIBRARY_OUTPUT = '../msdb/FS_hqtof.json'
# with open(FS_GNPS_LIBRARY_OUTPUT, "w") as f:
#     json.dump(sorted_list, f)

In [ ]:
HQTOF = functions_new.json_load('../msdb/FS_hqtof.json')
hqtof_search = FlashEntropySearch()
FS_SPECTRA = hqtof_search.build_index(HQTOF) # Pre-clean and sorted flash spectra

In [ ]:
len(FS_SPECTRA)

# Kdeplot

In [ ]:
def kde_density(X,Y):
    '''
    Pre-calculate density for kde plot
    '''
    kde = gaussian_kde(np.vstack([X, Y]), bw_method='silverman')
    x_grid, y_grid = np.mgrid[x.min():x.max():100j, y.min():y.max():100j]
    positions = np.vstack([x_grid.ravel(), y_grid.ravel()])
    density = kde(positions)
    return x_grid, y_grid, density

In [ ]:
width = 3.5
height = width / 1.618
# To take an overview of the ISDB and EDB
# Param: FS_E_LIBRARY,FS_IS0_LIBRARY,FS_IS1_LIBRARY,FS_IS2_LIBRARY,
# Return: kdeplot

## EDB

In [ ]:
t = time.time()
E_pm,E_npeak,E_entropy = [],[],[]
for SPECTRUM in tqdm(FS_E_LIBRARY,total = len(FS_E_LIBRARY)):
    E_pm.append(SPECTRUM['precursor_mz'])
    E_npeak.append(SPECTRUM['n_peak'])
    E_entropy.append(SPECTRUM['entropy'])
E_DF = pd.DataFrame({
    'entropy': E_entropy,
    'PM': E_pm,
    'NPEAK':E_npeak
})

In [ ]:
# Random sample
sample_size = 50000  # 选择一个合适的样本大小
sample_df = E_DF.sample(n=sample_size)

plt.figure(figsize=(width * 2, height * 2))
kde = sns.kdeplot(
    data=sample_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    # cut=0,
    cbar=True,
    legend=False,
    color = '#E84445'
)
plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/E_distribution.png', dpi=300, bbox_inches= 'tight')

In [ ]:
# Pre-calculate kde density
x_grid, y_grid, density = kde_density(E_entropy,E_pm)

# 绘制图形
plt.figure(figsize=(width * 3, height * 2))
contour = plt.contourf(x_grid, y_grid, density.reshape(x_grid.shape),cmap='Greys')
# contour.set_clim(vmin=0, vmax=0.005) 
plt.gcf().set_facecolor('white')  # set the figure background as white
plt.gca().set_facecolor('white')  # set the axis backgroud as white
plt.colorbar(label='Density')
# plt.xlabel('Entropy')
# plt.ylabel('PM')
# plt.ylim(0, 100)
plt.show()
print(f'Finished in {(time.time() - t) / 60:.2f} min')

## ISDB_e0

In [ ]:
# ISDB_e0
t = time.time()
IS0_pm,IS0_npeak,IS0_entropy = [],[],[]
for SPECTRUM in tqdm(FS_IS0_LIBRARY,total = len(FS_IS0_LIBRARY)):
    IS0_pm.append(SPECTRUM['precursor_mz'])
    IS0_npeak.append(SPECTRUM['n_peak'])
    IS0_entropy.append(SPECTRUM['entropy'])
IS0_DF = pd.DataFrame({
    'entropy': IS0_entropy,
    'PM': IS0_pm,
    'NPEAK':IS0_npeak
})

print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
sample_size = 50000  # 选择一个合适的样本大小
i0_df = IS0_DF.sample(n=sample_size, random_state=42)
width = 3.5
height = width / 1.618
plt.figure(figsize=(width*2 , height*2))
kde = sns.kdeplot(
    data=i0_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    cbar=True,
    # cut=0,
    color = '#1999B2'
)

plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/is0_distribution.png', dpi=300, bbox_inches= 'tight')

In [ ]:
# Pre-calculate kde density
x_grid, y_grid, density = kde_density(IS0_DF['entropy'].values,IS0_DF['PM'].values)

# 绘制图形
plt.figure(figsize=(width * 3, height * 2))
contour = plt.contourf(x_grid, y_grid, density.reshape(x_grid.shape),cmap='Greys')
contour.set_clim(vmin=0, vmax=0.005) 

plt.gcf().set_facecolor('white')  # set the figure background as white
plt.gca().set_facecolor('white')  # set the axis backgroud as white
plt.colorbar(label='Density')

plt.xlabel('Entropy')
plt.ylabel('PM')
plt.ylim(0, 1500)
plt.show()

## ISDB_e1

In [ ]:
# ISDB_e1
t = time.time()
IS1_pm,IS1_npeak,IS1_entropy = [],[],[]
for SPECTRUM in tqdm(FS_IS1_LIBRARY,total = len(FS_IS1_LIBRARY)):
    IS1_pm.append(SPECTRUM['precursor_mz'])
    IS1_npeak.append(SPECTRUM['n_peak'])
    IS1_entropy.append(SPECTRUM['entropy'])
IS1_DF = pd.DataFrame({
    'entropy': IS1_entropy,
    'PM': IS1_pm,
    'NPEAK':IS1_npeak
})


In [ ]:
sample_size = 50000  # 选择一个合适的样本大小
i1_df = IS1_DF.sample(n=sample_size)

plt.figure(figsize=(width*2 , height*2))
kde = sns.kdeplot(
    data=i1_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    # cut=0,
    cbar=True,
    legend=False,
    color = '#95BCE5'
)
plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/is1_distribution.png', dpi=300, bbox_inches= 'tight')

In [ ]:
# Pre-calculate kde density
x_grid, y_grid, density = kde_density(IS1_DF['entropy'].values,IS1_DF['PM'].values)

# 绘制图形
plt.contourf(x_grid, y_grid, density.reshape(x_grid.shape),cmap='Greys')
plt.gcf().set_facecolor('white')  # set the figure background as white
plt.gca().set_facecolor('white')  # set the axis backgroud as white
plt.colorbar(label='Density')
plt.xlabel('Entropy')
plt.ylabel('PM')
plt.show()
print(f'Finished in {(time.time() - t) / 60:.2f} min')

## ISDB_e2

In [ ]:
# ISDB_e2
t = time.time()
IS2_pm,IS2_npeak,IS2_entropy = [],[],[]
for SPECTRUM in tqdm(FS_IS2_LIBRARY,total = len(FS_IS2_LIBRARY)):
    IS2_pm.append(SPECTRUM['precursor_mz'])
    IS2_npeak.append(SPECTRUM['n_peak'])
    IS2_entropy.append(SPECTRUM['entropy'])
IS2_DF = pd.DataFrame({
    'entropy': IS2_entropy,
    'PM': IS2_pm,
    'NPEAK':IS2_npeak
})

In [ ]:
sample_size = 100000  # 选择一个合适的样本大小
i2_df = IS2_DF.sample(n=sample_size)

plt.figure(figsize=(width*2 , height*2))
kde = sns.kdeplot(
    data=i2_df,
    x='entropy',
    y='PM',
    fill=True,
    common_grid=True,
    common_norm=False,
    bw_adjust=2,
    # cut=0,
    cbar=True,
    legend=False,
    color = '#F39DA0'
)
plt.ylim(0, 1500)
plt.xlim(0, 6)
plt.savefig(f'../msdb/data/pic/is2_distribution.png', dpi=300, bbox_inches= 'tight')

In [ ]:
# Pre-calculate kde density
x_grid, y_grid, density = kde_density(IS2_DF['entropy'].values,IS2_DF['PM'].values)

# 绘制图形
plt.contourf(x_grid, y_grid, density.reshape(x_grid.shape),cmap='Greys')
plt.gcf().set_facecolor('white')  # set the figure background as white
plt.gca().set_facecolor('white')  # set the axis backgroud as white
plt.colorbar(label='Density')
plt.xlabel('Entropy')
plt.ylabel('PM')
plt.show()
print(f'Finished in {(time.time() - t) / 60:.2f} min')

# Accuracy of ISDB
## Inner consistency of ISDB
### Spectral similarity of ISDB_e0/e1/e2

In [ ]:
'''
Cluster the in-silico spectra to find high-accuracy spectra
Param: e_search,is0_search,is1_search,is2_search; FS_E_LIBRARY,FS_IS0_LIBRARY,FS_IS1_LIBRARY,FS_IS2_LIBRARY
Scripts: ICofIS.py
'''

In [ ]:
# np.save('../msdb/lib_matrix/test.npy',SPEC_IS0_SIM_MATRIC)
test_matrix= np.load('../msdb/lib_matrix/test.npy')

In [ ]:
icis0 = functions_new.json_load('../msdb/data/ICofIS/is0_Id2Smile.json')
icis1 = functions_new.json_load('../msdb/data/ICofIS/is1_Id2Smile.json')
icis2 = functions_new.json_load('../msdb/data/ICofIS/is2_Id2Smile.json')

### Chemical similarity matrices
`ICofIS_ChemSim.py`

In [ ]:
# Create fp databased by FPsim2
# Param: Index-to-SMILE, specific finger print type(Morgan_radius2_fpSize1024)
# return: fp database file
t = time.time() 
EL = 2 # Energy Lever
ID2SMILE = functions_new.json_load(f'../msdb/data/ICofIS/is{EL}_Id2Smile.json')
FPDB_FILE = f'../msdb/data/ICofIS/chem/is{EL}_fp_db.h5'
OUTPUT_MATRIX = f'../msdb/data/ICofIS/chem/is{EL}_ICofIS.npy'

IDX2SMILE = [[s['smile'],idx] for idx,s in enumerate(ID2SMILE)]

if not os.path.exists(FPDB_FILE):
    create_db_file(
        mols_source=IDX2SMILE, 
        filename=FPDB_FILE,
        mol_format='smiles', # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )
print(f'Finished in {(time.time() - t) / 60:.2f} min')


In [ ]:
# Calculate structural similarity pair >= specific threshold (Dice_radius2_0.75)
fpe = FPSim2Engine(FPDB_FILE)
hit_dict = {}
for idx,s in tqdm(enumerate(IDX2SMILE),total = len(IDX2SMILE)):
    SMILE = s[0]
    results = fpe.similarity(SMILE, threshold=0.75, metric='dice', n_workers=1)# [(LIBidx,similarity1),(),...]
    fpidx_list = [x[0] for x in results]
    fpsim_list = [x[1] for x in results]
    if fpidx_list:
        hit_dict[idx] = {'idx':fpidx_list,'sim_list':fpsim_list} # {LIBidx: {},{},..}

In [ ]:
# Save matrix
ECHEM_MATRIX  = np.full((len(IDX2SMILE),len(IDX2SMILE)), 0.0, dtype=float) 
for key, value in hit_dict.items():
    QueryIdx = key
    MatchIdxs = value['idx']
    MatchSim = value['sim_list']
    for idx, MatchIdx in enumerate(MatchIdxs):
        ECHEM_MATRIX[QueryIdx,MatchIdx] = MatchSim[idx]
np.save(OUTPUT_MATRIX,ECHEM_MATRIX)

### Clustering
`ICofIS_networking.py`

### Evaluation

In [ ]:
DIR = f'../msdb/data/ICofIS/'
IDXtoFID = {}
for idx,s in enumerate(FS_IS_LIBRARY):
    IDXtoFID[idx] = s['id']
CCMSIDtoIDX = {value: key for key, value in IDXtoFID.items()}
with open('../msdb/data/isdb/IDXtoFID.json','w') as f:
    json.dump(IDXtoFID,f)


ISID_SMILE = functions_new.json_load('../msdb/data/ICofIS/is0_Id2Smile.json')
ESPEC_MATRIX = np.load(f'../msdb/data/ICofIS/entropy_is{EL}_sim_icofis.npy')
EMP_MATRIX = np.load(f'../msdb/data/ICofIS/entropy_is{EL}_peak_icofis.npy')
ECHEM_MATRIX = np.load(f'../msdb/data/ICofIS/chem/is{EL}_ICofIS.npy')

In [ ]:
# Load search matrix
# THRESHOLDs = list(np.arange(0, 1.1, 0.001))
# peak_threshold = list(np.arange(1,11,1))

G1,G2,G3,G4 = topology.get_groups() # Load algorithms

SHARED_PEAKs = False
SEARCH_ALGORITHMs = ['entropy']
DIR = '../msdb/data/std/SearchMatrix/'

ISM_RES = []
for ALGO in tqdm(SEARCH_ALGORITHMs,total = len(SEARCH_ALGORITHMs)):
    # Load Spec search matrices:
    try:
        ESPEC_MATRIX = np.load(f'../msdb/data/ICofIS/entropy_is{EL}_sim_icofis.npy')
        EMP_MATRIX = np.load(f'../msdb/data/ICofIS/entropy_is{EL}_peak_icofis.npy')
        ECHEM_MATRIX = np.load(f'../msdb/data/ICofIS/chem/is{EL}_ICofIS.npy')
        
        if SHARED_PEAKs:
            EMIDXs = np.full(ESPEC_MATRIX.shape[0], 0, dtype=int)   
            EMaxSpecSim  = np.full(ESPEC_MATRIX.shape[0], 0.0, dtype=float) 
            EMaxChemSim = np.full(ESPEC_MATRIX.shape[0], 0.0, dtype=float)
            
            for idx,EMP_ARR in enumerate(EMP_MATRIX):
                cols = np.where(EMP_ARR >=6)[0] # <tuple> Min matched peaks >= 5
                if cols.any(): # 
                    FilColIdx = np.argmax(ESPEC_MATRIX[idx, cols])
                    best_col = cols[FilColIdx]
                    EMIDXs[idx] = best_col
                    EMaxSpecSim[idx]  =  ESPEC_MATRIX[idx, best_col]
                    EMaxChemSim[idx] = ECHEM_MATRIX[idx, best_col]
            ELables = [1 if x >= 0.75 else 0 for x in EMaxChemSim] # Chem sim ≥ 0.75 is considered true positive
            
            # ROC_AUC curve
            FPRs, TPRs, THRESHOLDs = roc_curve(ELables, EMaxSpecSim)
            AUC = auc(FPRs,TPRs)
            # J = TPRs - FPRs # # Youden's J to find the optimal threshold
            # optimal_idx = np.argmax(J)
            OT = optimal_threshold[ALGO] # optimal spectral similarity threshold 
            Y_PREDCTIONs = [1 if x >= OT else 0 for x in EMaxSpecSim] # Spectral sim ≥ threholds is considered as 1 
            TN, FP, FN, TP  = confusion_matrix(ELables, Y_PREDCTIONs).ravel()
            FDR = FP/(FP+TP) 

            
        else:
            # EMaxIdx = []
            
            EMaxIdx = [np.argmax(arr) for arr in ESPEC_MATRIX] # Top1 scoring
            EMaxSpecSim = [ESPEC_MATRIX[idx][EMaxIdx[idx]] for idx in range(len(EMaxIdx))] # Spectral similarity with top 1 match
            EMaxChemSim = [ECHEM_MATRIX[idx][EMaxIdx[idx]] for idx in range(len(EMaxIdx))] # Chemical dice simialrity with top 1 match
            ELables = [1 if x >= 0.75 else 0 for x in EMaxChemSim] 
        
            # ROC_AUC curve
            FPRs, TPRs, THRESHOLDs = roc_curve(ELables, EMaxSpecSim)
            AUC = auc(FPRs,TPRs)
            
            J = TPRs - FPRs # # Youden's J to find the optimal threshold
            optimal_idx = np.argmax(J)
            OT = THRESHOLDs[optimal_idx] # optimal spectral similarity threshold 
            Y_PREDCTIONs = [1 if x >= OT else 0 for x in EMaxSpecSim] # Spectral sim ≥ threholds is considered as 1 
            TN, FP, FN, TP  = confusion_matrix(ELables, Y_PREDCTIONs).ravel()
            FDR = FP/(FP+TP) 
        ISM_RES.append({'library':'in-silico',
                       'spectral_algorithm':ALGO,
                       'optimal_threshold': OT,
                        'TN':TN,'FP':FP,'FN':FN,'TP':TP,
                       'FDR':FDR,'TPR':TPRs[optimal_idx],'FPR':FPRs[optimal_idx],
                        'AUC':AUC
                       })
    except:print(ALGO)

ISM_DF = pd.DataFrame(ISM_RES).sort_values(by='TP', ascending=False)
ISM_DF.to_csv(f'../msdb/data/ICofIS/is{EL}_ICofIS.csv')


### PCA plot

In [ ]:
def fp2arr(fp):
    arr = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

In [ ]:
# small sample represent the total database
# icis0, icis1, icis2 <dict> FS_IS0_LIBRARY
class1=[]
molecules=[]
ecfp_1024=[]
X = []
CLASS1 = 'class1'
CLASS2 = 'class2'
for i in trange(len(icis0)):
    try:
        SMILE = icis0[i]['smile']
        mol = Chem.MolFromSmiles(SMILE)
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
        class1.append(CLASS1)
        X.append(np.array(fp2arr(ecfp)))
    except:
        pass

for i in trange(len(FS_IS0_LIBRARY)):
    try:
        SMILE = FS_IS0_LIBRARY[i]['smile']
        mol = Chem.MolFromSmiles(SMILE)
        ecfp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
        class1.append(CLASS2)
        X.append(np.array(fp2arr(ecfp)))
    except:
        pass


pca = PCA()
pca.fit(X)
reduced_X = pca.transform(X)

# Access the explained variance ratio of each principal component
explained_var_ratio = pca.explained_variance_ratio_

# Print the explained variance ratio of each principal component
pc1 = ''
pc2 = ''
pc3 = ''
for i, var_ratio in enumerate(explained_var_ratio):
    print(f'Principal Component {i + 1}: Explained Variance Ratio = {var_ratio:.4f}')
    if i+1 == 1:
        pc1 = f'{(var_ratio*100):.2f}%'
    if i+1 == 2:
        pc2 = f'{(var_ratio*100):.2f}%'
    if i+1 == 3:
        pc3 = f'{(var_ratio*100):.2f}%'
    if i+1 > 2:break

print(pc1,pc2,pc3)
red_x, red_y, red_z= [], [], []
blue_x, blue_y, blue_z = [], [], []
green_x, green_y, green_z = [], [], []

for i in trange(len(reduced_X)):
    if class1[i] == CLASS1:
        red_x.append(reduced_X[i][0])
        red_y.append(reduced_X[i][1])

    elif class1[i] == CLASS2:
        blue_x.append(reduced_X[i][0])
        blue_y.append(reduced_X[i][1])
  


fig = plt.figure(dpi=150)
ax = fig.add_subplot(111)
ax.scatter(red_x, red_y, c='#1A51AD', marker='o', label='selected',alpha=0.5)
ax.scatter(blue_x, blue_y, c='gray', marker='o', label='ISDB',alpha=0.015)
ax.legend()
ax.set_xlabel(f'PC1 ({pc1})')
ax.set_ylabel(f'PC2 ({pc2})')
ax.set_title('2D PCA')

# plt.savefig('scaffolds.pdf', format='pdf')
plt.show()

## hqtof search against IS
### Non-redundant GNPS library

In [ ]:
# Filter and retain unique compounds with smiles (47193)
SMILEs = []
CUR_FS_E_LIBRARY = []
for SPEC in tqdm(FS_E_LIBRARY,total=len(FS_E_LIBRARY)):
    SMILE = SPEC['smile']
    if SMILE not in SMILEs:
        SMILEs.append(SMILE)
        CUR_FS_E_LIBRARY.append(SPEC)
        
# Save as json file
for SPEC in tqdm(CUR_FS_E_LIBRARY,total = len(CUR_FS_E_LIBRARY)):
    PEAKS = SPEC['peaks'] 
    SPEC['peaks'] = PEAKS.tolist()

FS_GNPS_LIBRARY_OUTPUT0 = '../msdb/FS_edb_nr.json'
with open(FS_GNPS_LIBRARY_OUTPUT0, "w") as f:
    json.dump(CUR_FS_E_LIBRARY, f)

In [ ]:
# Load filtered library
t = time.time()
FS_E_LIBRARY_NR = functions_new.json_load('../msdb/FS_hqtof.json')
e_nr_search = FlashEntropySearch()
FS_E_LIBRARY_NR = e_nr_search.build_index(FS_E_LIBRARY_NR)
print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
## Match against the ISDB e0/e1/e2 by modified cosine and entropy
## EmIS.py
EmI_SIM_MATRIC = np.empty((0, len(FS_IS1_LIBRARY)))
EmI_IS_NPEAK_MATRIC = np.empty((0, len(FS_IS1_LIBRARY)))
for SPEC in tqdm(FS_E_LIBRARY_NR,total = len(FS_E_LIBRARY_NR)):
    PM = SPEC['precursor_mz']
    PEAKs = SPEC['peaks']
    
    SIM_ARRAY = is1_search.identity_search(
        precursor_mz = PM,
        peaks = PEAKs,
        ms1_tolerance_in_da = 0.02,
        ms2_tolerance_in_da = 0.02,
        output_matched_peak_number = False)
        
    EmI_SIM_MATRIC = np.vstack((EmI_SIM_MATRIC,SIM_ARRAY))

### Chemical similarity

In [ ]:
# h_emis:h qtof (2538, 454887)
# Create fp databased by FPsim2
# Param: Index-to-SMILE, specific finger print type(Morgan_radius2_fpSize1024)
# return: fp database file
t = time.time() 
DIR = f'../msdb/data/ICofIS/'
EL = 2 # Energy Level 0,1,2
FPDB_FILE = os.path.join(DIR,f'chem/is{EL}_fp_db_emis.h5')



OUTPUT_MATRIX = f'../msdb/data/ICofIS/chem/is{EL}_emis.npy'

IDX2SMILE = [[s['smile'],idx] for idx,s in enumerate(FS_IS2_LIBRARY)]

if not os.path.exists(FPDB_FILE):
    create_db_file(
        mols_source=IDX2SMILE, 
        filename=FPDB_FILE,
        mol_format='smiles', # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )
print(f'Finished in {(time.time() - t) / 60:.2f} min')


In [ ]:
# Calculate structural similarity pair >= specific threshold (Dice_radius2_0.75)
# FS_SPECTRA: hqtof
fpe = FPSim2Engine(FPDB_FILE)
hit_dict = {}
for idx,s in tqdm(enumerate(FS_SPECTRA),total = len(FS_SPECTRA)):
    SMILE = s['smile']
    results = fpe.similarity(SMILE, threshold=0.75, metric='dice', n_workers=1)# [(LIBidx,similarity1),(),...]
    fpidx_list = [x[0] for x in results]
    fpsim_list = [x[1] for x in results]
    if fpidx_list:
        hit_dict[idx] = {'idx':fpidx_list,'sim_list':fpsim_list} # {LIBidx: {},{},..}

In [ ]:
# Save matrix
ECHEM_MATRIX  = np.full((len(FS_SPECTRA),len(IDX2SMILE)), 0.0, dtype=float) 
for key, value in hit_dict.items():
    QueryIdx = key
    MatchIdxs = value['idx']
    MatchSim = value['sim_list']
    for idx, MatchIdx in enumerate(MatchIdxs):
        ECHEM_MATRIX[QueryIdx,MatchIdx] = MatchSim[idx]
np.save(OUTPUT_MATRIX,ECHEM_MATRIX)

### Evaluation


### Match against the ISDB

In [ ]:
CUR_SIM_MATRIX = []
CUR_NPEAK_MATRIX = []
for SPEC in tqdm(FS_E_LIBRARY_NR,total = len(FS_E_LIBRARY_NR)):
    PM = SPEC['precursor_mz']
    PEAKs = SPEC['peaks']
    SIM_ARRAY, NPEAK_ARRAY = is1_search.open_search(
    precursor_mz=PM, peaks=PEAKs,
    ms1_tolerance_in_da=0.02, ms2_tolerance_in_da=0.02,
    output_matched_peak_number=True)
    SIM_ARRAY = np.round(SIM_ARRAY.astype(float), decimals=2)
    CUR_SIM_MATRIX.append(SIM_ARRAY)
    CUR_NPEAK_MATRIX.append(NPEAK_ARRAY)

## Similarity between in-silico and experimental spectra

In [ ]:
# Duplicate
temp = functions_new.json_load('../msdb/FS_edb_smi.json')
e_search = FlashEntropySearch()
FS_E_LIBRARY = e_search.build_index(temp)

In [111]:
dup_E_LIBRARY = list({s['smile']: s for s in FS_E_LIBRARY}.values()) # Retain spectra with non-redunant similes
ISDB_list = list(ISDB_INFO) # Keys of ISDB_INFO <list>

In [80]:
# gnps_is:h qtof (47192, 467124)
# Create fp databased by FPsim2
# Param: Index-to-SMILE, specific finger print type(Morgan_radius2_fpSize1024)
# return: fp database file
t = time.time() 
DIR = f'../msdb/data/ICofIS/'
FPDB_FILE = os.path.join(DIR,f'chem/fp_db_gnps_is.h5')
OUTPUT_MATRIX = os.path.join(DIR,f'chem/gnps_is.npy')

IDX2SMILE = []
for idx, (key, values) in enumerate(ISDB_INFO.items()):
    IDX2SMILE.append([values['smiles'],idx])

if not os.path.exists(FPDB_FILE):
    create_db_file(
        mols_source=IDX2SMILE, 
        filename=FPDB_FILE,
        mol_format='smiles', # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )
print(f'Finished in {(time.time() - t) / 60:.2f} min')


[00:25:01] Explicit valence for atom # 20 N, 5, is greater than permitted
[00:27:14] Explicit valence for atom # 24 N, 5, is greater than permitted


Finished in 3.45 min


In [84]:
# Find identical GNPS-smile in isdb
# Calculate structural similarity pair >= specific threshold (Dice_radius2_1)
# FS_SPECTRA: hqtof
fpe = FPSim2Engine(FPDB_FILE)
hit_dict = {}
for idx,s in enumerate(dup_E_LIBRARY):
    SMILE = s['smile']
    results = fpe.similarity(SMILE, threshold=1, metric='dice', n_workers=1)# [(LIBidx,similarity1),(),...]
    fpidx_list = [x[0] for x in results]
    fpsim_list = [x[1] for x in results]
    if fpidx_list:
        hit_dict[idx] = {'idx':fpidx_list,'sim_list':fpsim_list} # {LIBidx: {},{},..}

In [120]:
# 18983 gnps-smiles found identical match in isdb
ALGO = 'entropy'
for key, values in hit_dict.items():
    dup_E_SPECTRUM = dup_E_LIBRARY[key]
    PM = dup_E_SPECTRUM['precursor_mz']
    SPEC = dup_E_SPECTRUM['peaks']
    SPEC = spectral_entropy.clean_spectrum(SPEC, max_mz=PM+0.01) # MS2 spectrum clean by normalizing and removing signals with intensity less than 1% of the base peak
    SPECTRUM = Spectrum(mz=np.array(SPEC[:, 0],dtype = float),
                  intensities=np.array(SPEC[:, 1],dtype = float),
                  metadata={"precursor_mz": PM})
    
    
    ISDB_idx = values['idx'][0]
    ISDB_ID = ISDB_list[ISDB_idx]
    e0_ms2,e0_spectrum,e1_ms2,e1_spectrum,e2_ms2,e2_spectrum = functions_new.get_spectra_from_isinfo(ISDB_INFO,ISDB_ID)

    SIM0, MP0 = functions_new.clac_spec_sim(SPEC,SPECTRUM,e0_ms2,e0_spectrum,ALGO)
    SIM1, MP1 = functions_new.clac_spec_sim(SPEC,SPECTRUM,e1_ms2,e1_spectrum,ALGO)
    SIM2, MP2 = functions_new.clac_spec_sim(SPEC,SPECTRUM,e2_ms2,e2_spectrum,ALGO)

    dup_E_SPECTRUM[f'{ALGO}_sim0'] = round(SIM0,2)
    dup_E_SPECTRUM[f'{ALGO}_mp0'] = MP0
    dup_E_SPECTRUM[f'{ALGO}_sim1'] = round(SIM1,2)
    dup_E_SPECTRUM[f'{ALGO}_mp1'] = MP1
    dup_E_SPECTRUM[f'{ALGO}_sim2'] = round(SIM2,2)
    dup_E_SPECTRUM[f'{ALGO}_mp2'] = MP2

In [131]:
dup_E_LIBRARY[19]
for r in dup_E_LIBRARY:
    print(r)
    break

{'id': 'CCMSLIB00000427077', 'precursor_mz': 32.05, 'peaks': array([[30.,  1.]], dtype=float32), 'smile': 'CN', 'charge': 1, 'ion_mode': 'positive'}


In [203]:
ALGO = 'peak_percentage'
keys = [f'{ALGO}_sim0',f'{ALGO}_mp0',
        f'{ALGO}_sim1',f'{ALGO}_mp1',
        f'{ALGO}_sim2',f'{ALGO}_mp2']
stats = {keys[0]:[],keys[1]:[],keys[2]:[],keys[3]:[],keys[4]:[],keys[5]:[]}
for SPECTRUM in dup_E_LIBRARY:
    for key in keys:
        try:
            stats[key].append(SPECTRUM[key])
        except:
            pass
idx1,idx3,idx5 = 0,2,4
total = len(stats[keys[idx1]])
print(f'{len([x for x in stats[keys[idx5]] if x >= 0.6])/total}% pairs higher than 0.6')
print(f'{len([x for x in stats[keys[idx1]] if 0.7 > x >= 0.35])/total}% pairs higher than 0.35 and lower than 0.6')
print(f'{len([x for x in stats[keys[idx1]] if 0.35 > x])/total}% pairs lower than 0.35')

0.09993151767370806% pairs higher than 0.6
0.276563240794395% pairs higher than 0.35 and lower than 0.6
0.6691250065848391% pairs lower than 0.35


In [194]:
# 创建一个共享 x 轴的双 y 轴图
plt.rcParams['font.sans-serif'] = ['Arial'] 
fig, ax1 = plt.subplots(figsize=(3, 6))
idx1,idx2 = 4,5
x_ls = np.zeros(len(stats[keys[idx1]]))      # 全 0
x_hs = np.ones(len(stats[keys[idx2]]))       # 全 1

# 绘制 LS_values 的小提琴图
# sns.violinplot(y=stats[keys[0]], color='blue', inner=None, ax=ax1, width=0.5)
sns.violinplot(x=x_ls, y=stats[keys[idx1]],
               color='#F5A889', inner=None, width=0.4, ax=ax1)
ax1.set_title(f"{keys[idx1]}")
ax1.set_ylabel("Spectral similarity", color='black')
ax1.tick_params(axis='y', labelcolor='black')

# for spine in ax1.spines.values(): # 隐藏 ax1 的边框
#     spine.set_visible(False)
ax1.xaxis.set_ticks_position('none')  # 隐藏 x 轴刻度
ax1.yaxis.set_ticks_position('none')  # 隐藏 y 轴刻度

ax2 = ax1.twinx() # 创建第二个 y 轴
ax2.spines['right'].set_position(('outward', -3)) # 调整 ax2 的位置，使其更靠近 ax1

# 绘制 HS_values 的小提琴图
sns.violinplot(x=x_hs, y=stats[keys[idx2]],
               color='#ACD6EC', inner=None, width=0.4, ax=ax2)
ax2.set_ylabel("Shared peaks", color='black')
ax2.tick_params(axis='y', labelcolor='black')

# 隐藏 ax2 的边框
# for spine in ax2.spines.values():
#     spine.set_visible(False)

plt.tight_layout()
plt.savefig(os.path.join(DIR,f'pic/violion_{keys[idx1]}.png'), dpi=500, bbox_inches="tight")
# plt.show()
plt.close()

# Hqtof spectral sim matrix
1. Spectral simialrity matrices

2. Structural similarity matrices

* Tanimoto
* Maximum Common Substructure jaccard oefficient (MCS)
    

## Spectral similarity 

In [ ]:
t = time.time()
hqtof_pair = FlashEntropySearchCore()
(e_ions_mz_idx_start,e_ions_mz,e_ions_intensity,e_ions_spec_idx,
e_nl_mass_idx_start,e_nl_mass,e_nl_intensity,e_nl_spec_idx,e_ions_idx_for_nl,
) = hqtof_pair.build_index(FS_SPECTRA)
print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
# Get Compare_dict
t = time.time()
# Param
# Stack lib ions
compare_dict = 
print(f'Finished in {(time.time() - t) / 60:.2f} min')

In [ ]:
# Save as pickle file
with open('../msdb/data/hqtof/compare_dict.pkl', 'wb') as f:
    pickle.dump(compare_dict, f)

In [ ]:
# Load pickle file
with open('../msdb/data/hqtof/compare_dict.pkl', 'rb') as f:
    compare_dict = pickle.load(f)

In [ ]:
G1,G2,G3,G4 = topology.get_groups()
ALGOs = G1 + G2 + G3 + G4 
SEARCH_ALGOs = [sa for sa in ALGOs if sa not in ['modified_cosine','entropy','neutral_loss']]
compare_dict = functions_new.json_load('../msdb/data/hqtof/compare_dict.json')

In [ ]:
# Search
t = time.time()
SEARCH_ALGOs = ['entropy']

cosine = CosineGreedy(tolerance=0.02)
neutral_loss = NeutralLossesCosine(0.02)
modified_cosine = ModifiedCosine(tolerance=0.02)

for SEARCH_ALGO in SEARCH_ALGOs:
    print(SEARCH_ALGO)
    EMS2_RESULT = {}
    EMS2_SIM_MATRIX = np.zeros((len(FS_SPECTRA), len(FS_SPECTRA)), dtype=float) # Initialize empty 
    EMS2_PEAK_MATRIX = np.zeros((len(FS_SPECTRA), len(FS_SPECTRA)), dtype=float)
                                 
    for query_idx, E_idxs in tqdm(compare_dict.items(),total = len(compare_dict)):
        query_PM = FS_SPECTRA[query_idx]['precursor_mz']
        query_PEAKS = FS_SPECTRA[query_idx]['peaks']

        spectrum_1 = Spectrum(mz=np.array(query_PEAKS[:, 0],dtype = float),
                              intensities=np.array(query_PEAKS[:, 1],dtype = float),
                              metadata={"precursor_mz": query_PM})
        
        temp_SIM_ARR = np.zeros((1, len(FS_SPECTRA))) 
        temp_PEAK_ARR = np.zeros((1, len(FS_SPECTRA)))
        for E_idx in E_idxs: # E_idxs
            E_PM = FS_SPECTRA[E_idx]['precursor_mz']
            E_PEAKS = FS_SPECTRA[E_idx]['peaks']
            spectrum_2 = Spectrum(mz=np.array(E_PEAKS[:, 0],dtype = float),
                                  intensities=np.array(E_PEAKS[:, 1],dtype = float),
                                  metadata={"precursor_mz": E_PM})

            if SEARCH_ALGO == 'modified_cosine':
                    
                score = modified_cosine.pair(spectrum_1, spectrum_2)
                SPEC_SIM = score['score']
                N_PEAK = score['matches']
    
                EMS2_SIM_MATRIX[query_idx,E_idx] = SPEC_SIM
                EMS2_PEAK_MATRIX[query_idx,E_idx] = N_PEAK

            elif SEARCH_ALGO == 'cosine':
                score = cosine.pair(spectrum_1, spectrum_2)
                
                SPEC_SIM = score['score']
                N_PEAK = score['matches']
        
                EMS2_SIM_MATRIX[query_idx,E_idx] = SPEC_SIM
                EMS2_PEAK_MATRIX[query_idx,E_idx] = N_PEAK  
                
            elif SEARCH_ALGO == 'neutral_loss':
                score = neutral_loss.pair(spectrum_1, spectrum_2)
                
                SPEC_SIM = score['score']
                N_PEAK = score['matches']
        
                EMS2_SIM_MATRIX[query_idx,E_idx] = SPEC_SIM
                EMS2_PEAK_MATRIX[query_idx,E_idx] = N_PEAK   
            
            # # if functions_new.calculate_ppm(query_PM,E_PM) <= 10:
            else:
                    
                SPEC_SIM = similarity(query_PEAKS, E_PEAKS, method=SEARCH_ALGO, ms2_da=0.02)  
                EMS2_SIM_MATRIX[query_idx,E_idx] = SPEC_SIM

        
    np.save(f'../msdb/data/hqtof/SpecSimMatrix/{SEARCH_ALGO}.npy',EMS2_SIM_MATRIX)
    np.save(f'../msdb/data/hqtof/SpecSimMatrix/{SEARCH_ALGO}_mp.npy',EMS2_PEAK_MATRIX)

print(f'Finished in {(time.time() - t) / 60:.2f} min')

## Pairwise calculation

In [ ]:
def get_spectra_from_einfo(EINFO,ID):
    '''

    :param einfo:
    :param id:
    :return:
    '''
    edb_pm = float(EINFO[ID]['PEPMASS'])
    ESMILE = EINFO[ID]['SMILE']
    temp = EINFO[ID]['MS2_SPECTRUM'] # <list>[[],[],...]
    if type(temp) == str:
        ESPEC = np.asarray(ast.literal_eval(temp)) # "[[],[],[],...]"
    else:
        ESPEC = temp
    ESPEC = spectral_entropy.clean_spectrum(ESPEC, max_mz=edb_pm+0.01)
    ESPECTRUM = Spectrum(mz=np.array(ESPEC[:, 0],dtype = float),
                  intensities=np.array(ESPEC[:, 1],dtype = float),
                  metadata={"precursor_mz": edb_pm,"smile":ESMILE})

    return ESPEC,ESPECTRUM

In [ ]:
# Load data
DIR = f'../msdb/data/hqtof/' # former: f'../msdb/data/hqtof/matrix/'
Hqtof_CCMSIDs = list(np.load(os.path.join(DIR,'idlist/H_qtof_non-redundant_CCMSIDs.npy'))) # [CCMSID1,CCMSID2,CCMSID3, ...]
GNPS_INFO = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')


In [ ]:
# Re-calculate by update matchms package
ALGO = 'modified_cosine'
SPEC_SIM_MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGO}.npy')) # 
MP_MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/cosine_peak_matrix.npy')) # 

# SPEC1,SPECTRUM1 = get_spectra_from_einfo(GNPS_INFO,'CCMSLIB00000001547')
# SPEC2,SPECTRUM2 = get_spectra_from_einfo(GNPS_INFO,'CCMSLIB00000001548')                                    
# SPEC_SIM,MP = functions_new.clac_spec_sim(SPEC1,SPECTRUM1, SPEC2,SPECTRUM2,ALGO)

In [ ]:
# Load 
DIR = f'../msdb/data/hqtof/' # former: f'../msdb/data/hqtof/matrix/'
Hqtof_CCMSIDs = list(np.load(os.path.join(DIR,'idlist/H_qtof_non-redundant_CCMSIDs.npy'))) # [CCMSID1,CCMSID2,CCMSID3, ...]
GNPS_INFO = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

MP_MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/cosine.npy')) # 
SPEC_SIM_MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/cosine.npy')) # 

# Calc 
ALGO = 'modified_cosine'
for idx1 in trange(len(Hqtof_CCMSIDs)):
    CCMSID1 = Hqtof_CCMSIDs[idx1]
    SPEC1,SPECTRUM1 = get_spectra_from_einfo(GNPS_INFO,CCMSID1)

    for idx2 in range(idx1):
        CCMSID2 = Hqtof_CCMSIDs[idx2]
        SPEC2,SPECTRUM2 = get_spectra_from_einfo(GNPS_INFO,CCMSID2)
        if SPEC_SIM_MATRIX1[idx1,idx2] > 0:
            SPEC_SIM,MP = functions_new.clac_spec_sim(SPEC1,SPECTRUM1, SPEC2,SPECTRUM2,ALGO)
            SPEC_SIM_MATRIX1[idx1,idx2] = SPEC_SIM
            SPEC_SIM_MATRIX1[idx2,idx1] = SPEC_SIM
            MP_MATRIX1[idx1,idx2] = MP
            MP_MATRIX1[idx2,idx1] = MP
        

In [ ]:
t = time.time()
# parse args
# args = parse_args()
# ALGORITHM = args.algorithm
ALGORITHM = 'cosine'

# Load processed json file
GNPS_JSON = '../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json'
with open(GNPS_JSON, 'r') as f:
    gnps_info = json.load(f)

# # Load filtered CCMSIDs
# CCMSIDs = np.load('H_qtof_non-redundant_CCMSIDs.npy').tolist()

In [ ]:
# Calculate matrices
# Create empty matrices
similarity_matrix = np.zeros((len(CCMSIDs), (len(CCMSIDs))))
peak_matrix = np.zeros((len(CCMSIDs), (len(CCMSIDs))))

# Calculate by different algorithms
for idx1 in trange(len(CCMSIDs)):
    spec1, spectrum1 = functions_new.GNPS_info_format(gnps_info, CCMSIDs[idx1])
    for idx2 in range(idx1):
        spec2, spectrum2 = functions_new.GNPS_info_format(gnps_info, CCMSIDs[idx2])
        if ALGORITHM == 'cosine':
            try:
                RESULT = peaktools.cosine(spectrum1, spectrum2, 0.05)
                SIMILARITY = round(RESULT.score, 2)
                N_PEAKS = RESULT.matches
            except ZeroDivisionError:
                SIMILARITY, N_PEAKS = 0, 0

        if ALGORITHM == 'modified_cosine':
            try:
                RESULT = peaktools.modified_cosine(spectrum1, spectrum2, 0.05)
                SIMILARITY = round(RESULT.score,2)
                N_PEAKS = RESULT.matches
            except ZeroDivisionError:
                SIMILARITY, N_PEAKS = 0, 0

        if ALGORITHM == 'neutral_loss':
            try:
                RESULT = peaktools.neutral_loss(spectrum1, spectrum2, 0.05)
                SIMILARITY = round(RESULT.score, 2)
                N_PEAKS = RESULT.matches
            except ZeroDivisionError:
                SIMILARITY, N_PEAKS = 0, 0

        else:
            try:
                SIMILARITY = round(spectral_entropy.similarity(spec1, spec2, method=ALGORITHM, ms2_da=0.05), 2)

            except ZeroDivisionError:
                SIMILARITY, N_PEAKS = 0, 0

        similarity_matrix[idx1, idx2] = SIMILARITY
        similarity_matrix[idx2, idx1] = SIMILARITY
        peak_matrix[idx1, idx2] = N_PEAKS
        peak_matrix[idx2, idx1] = N_PEAKS

In [ ]:
# Save similarity matrices
np.save(f'{ALGORITHM}_similarity_matrix.npy', similarity_matrix)
np.save(f'{ALGORITHM}_peak_matrix.npy', peak_matrix)

# Save dictinary (indices <str> : CCMSIDs)
INDEXtoCCMSID = {index: CCMSID for index, CCMSID in enumerate(CCMSIDs)}
with open (f'{ALGORITHM}_INDEXtoCCMS.json', 'w') as f2:
    json.dump(INDEXtoCCMSID,f2)

print(f'Finished in {(time.time() - t) / 60:.2f} min')

## Chemical similarity
* Dice similarity, n_bits = 1024, radius = 2

In [ ]:
# Create FP database
IDX2SMILE = []

for idx, SPEC in tqdm(enumerate(Hqtof_CCMSIDs), total=len(Hqtof_CCMSIDs)):
    CCMSID = Hqtof_CCMSIDs[idx]
    SMILE = GNPS_INFO[CCMSID]['SMILE']
    IDX2SMILE.append([SMILE, idx])

FPDB = f'../msdb/data/hqtof/chem/hqtof_fp_db.h5'
if not os.path.exists(FPDB):
    create_db_file(
        mols_source=IDX2SMILE,
        filename=FPDB,
        mol_format='smiles',  # required
        fp_type='Morgan',
        fp_params={'radius': 2, 'fpSize': 1024}
    )

In [ ]:
# FP_SPECTRA search against FS_E_LIB
fpe = FPSim2Engine(fp_filename=FPDB)
hit_dict = {}
for idx in trange(len(Hqtof_CCMSIDs)):
    try:
        CCMSID = Hqtof_CCMSIDs[idx]
        SMILE = GNPS_INFO[CCMSID]['SMILE']
        results = fpe.similarity(SMILE, threshold=0, metric='dice', n_workers=1)  # [(LIBidx,similarity1),(),...]
        results = [x for x in results if x[0] != idx] # Remove self comparison
        fpidx_list = [x[0] for x in results]
        fpsim_list = [x[1] for x in results]

        if fpidx_list:
            hit_dict[idx] = {'idx': fpidx_list, 'sim_list': fpsim_list}  # {LIBidx: {},{},..}
    except:print(idx)
    

In [ ]:
ECHEM_MATRIX  = np.full((len(Hqtof_CCMSIDs),len(Hqtof_CCMSIDs)), 0.0, dtype=float) 
for key, value in hit_dict.items():
    QueryIdx = key
    MatchIdxs = value['idx']
    MatchSim = value['sim_list']
    for idx, MatchIdx in enumerate(MatchIdxs):
        ECHEM_MATRIX[QueryIdx,MatchIdx] = MatchSim[idx]

# Save results
DIR = '../msdb/data/hqtof/chem/'
np.save(os.path.join(DIR,f'hqtof_dice.npy'),ECHEM_MATRIX)


## Peak percentage similairty

In [ ]:
peak_matrix = np.zeros((len(IDXtoCCMSID), (len(IDXtoCCMSID))))
for idx1 in trange(len(IDXtoCCMSID)):
    CCMSID = IDXtoCCMSID[str(idx1)]
    spec,spectrum = functions_new.GNPS_info_format(gnps_info,CCMSID)
    total_peaks = len(spec)
    for idx2 in range(idx1):
        n_peaks = N_PEAK_MATRIX[idx1,idx2]
        try:
            peak_percentage = n_peaks/total_peaks
        except:peak_percentage = 0
        peak_matrix[idx1,idx2]  = peak_percentage
        peak_matrix[idx2, idx1] = peak_percentage
np.save(f'../msdb/data/matrix/peak_percentage_similarity_matrix.npy', peak_matrix)

# Preprocess matrices

## Structural matrices

In [ ]:
# Load gnps-library json
gnps_info = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

# Load structural matrices
TANIMOTO_MATRIX_FILE = '../msdb/data/matrix/tanimoto_similarity_matrix.npy'
TANIMOTO_MATRIX_INDEX = '../msdb/data/matrix/tanimoto_INDEXtoCCMS.json'

tanimoto_matrix = np.load(TANIMOTO_MATRIX_FILE)
tanimoto_index_info = functions_new.json_load(TANIMOTO_MATRIX_INDEX)

In [ ]:
# Check structural matrix (if one-to-one)
idx1,idx2 = 0,1

SMILE1 = gnps_info[tanimoto_index_info[f'{idx1}']]['SMILE']
SMILE2 = gnps_info[tanimoto_index_info[f'{idx2}']]['SMILE']

morgen_generator = cheminfo_tools.get_morgan_generator()
taniscore = TanimotoSimilarity(morgen_generator.GetFingerprint(Chem.MolFromSmiles(SMILE1))
                               ,morgen_generator.GetFingerprint(Chem.MolFromSmiles(SMILE2)))

tanimoto_matrix[idx1,idx2],taniscore 

## Spectral matrices

In [ ]:
# Load gnps-library json
gnps_info = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

# Load spectral matrices
ALGORITHM = 'cosine'
spec_sim_matrix = np.load(f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy')
n_peaks_matrix = np.load(f'../msdb/data/matrix/{ALGORITHM}_peak_matrix.npy')
spec_index_info = functions_new.json_load(f'../msdb/data/matrix/{ALGORITHM}_INDEXtoCCMS.json')


In [ ]:
# Check structural matrix (if one-to-one)
idx1,idx2 = 0,1
CCMSID1,CCMSID2 = IDXtoCCMSID[f'{idx1}'],IDXtoCCMSID[f'{idx2}']


spec1,spectrum1 = functions_new.GNPS_info_format(GNPS_INFO,CCMSID1) 
spec2,spectrum2 = functions_new.GNPS_info_format(GNPS_INFO,CCMSID2)

result = peaktools.cosine(spectrum1,spectrum2,0.05)
# result.score, result.matches, spec_sim_matrix[idx1,idx2], n_peaks_matrix[idx1,idx2]

result,result[0],result[1]

In [ ]:
from my_packages import evaluation

group1,group2,group3,group4 = evaluation.get_groups()
for a in group1:
    print(a)

## Peak matrices

In [ ]:
# Create peak percentage matrix
ALGORITHM = 'modified_cosine'
if ALGORITHM == 'neutral_loss':
    N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_peak_matrix.npy'  # expect for modified_cosine and neutral_loss
elif ALGORITHM == 'modified_cosine':
    N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_peak_matrix.npy'
else:
    N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/cosine_peak_matrix.npy'
N_PEAK_MATRIX = np.load(N_PEAK_MATRIX_FILE)

CCMSID1 = IDXtoCCMSID[str(0)]
SPEC1,SPECTRUM1 = functions_new.GNPS_info_format(GNPS_INFO,CCMSID1)

SIM_MATRIX = np.zeros((len(N_PEAK_MATRIX), (len(N_PEAK_MATRIX))))
for idx1 in trange(len(N_PEAK_MATRIX)):
    CCMSID1 = IDXtoCCMSID[str(idx1)]
    SPEC1,SPECTRUM1 = functions_new.GNPS_info_format(GNPS_INFO,CCMSID1)
    N_PEAKs = len(SPEC1)
    for idx2 in range(idx1):
        SHARED_PEAKs = N_PEAK_MATRIX[idx1,idx2]
        if N_PEAKs !=0:
            percentage = round(SHARED_PEAKs/N_PEAKs,2)
        else:
            percentage = 0

        SIM_MATRIX[idx1,idx2] = percentage
        SIM_MATRIX[idx2,idx1] = percentage
        
np.save(f'../msdb/data/matrix/{ALGORITHM}MCS_similarity_matrix.npy', SIM_MATRIX)      

# MS1 match matrices

In [ ]:
GNPS_INFO = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')
CCMSIDs = np.load('../msdb/data/idlist/H_qtof_non-redundant_CCMSIDs.npy')


MS1_MATCH_MATRIX = np.zeros((len(CCMSIDs), (len(CCMSIDs))))
for idx1 in trange(len(CCMSIDs)):
    CCMSID1 = CCMSIDs[idx1]
    PM1 = float(GNPS_INFO[CCMSID1]['PEPMASS'])
    for idx2 in range(idx1):
        CCMSID2 = CCMSIDs[idx2]
        PM2 = float(GNPS_INFO[CCMSID2]['PEPMASS'])
        PPM = functions_new.calculate_ppm(PM1,PM2)
        if PPM < 15:
            MS1_MATCH_MATRIX[idx1,idx2] = 1
            MS1_MATCH_MATRIX[idx2, idx1] = 1
        else:
            MS1_MATCH_MATRIX[idx1, idx2] = 0
            MS1_MATCH_MATRIX[idx2, idx1] = 0

np.save(f'../msdb/data/structural_matrix/MS1match_matrix.npy', MS1_MATCH_MATRIX)

# Network evaluation
## Chemical similarity
还是有ccms没有class，需要check一下

In [ ]:
# Networking
G = nx.MultiGraph()
THRESHOLD = 0.7
for idx1 in trange(len(IDXtoCCMSID)):
    CCMSID = IDXtoCCMSID[f'{idx1}']
    G.add_node(CCMSID)
    # node_attr = {'pepmass': pm1}
    # G.add_node(id1, **node_attr)  # add nodes and attributes
    
for idx1 in trange(len(IDXtoCCMSID)):
    CCMSID1 = IDXtoCCMSID[str(idx1)]
    for idx2 in range(idx1):
        CCMSID2 = IDXtoCCMSID[str(idx2)]
        sim = TANIMOTO_MATRIX [idx1,idx2]
        if sim >= 0.7:
            edge_attr = {'tanimoto_similarity': sim, 'edge_type': 'tanimoto'}
            G.add_edge(CCMSID1, CCMSID2, **edge_attr)

MN_output = f'../msdb/data/network/MN_tanimoto_{THRESHOLD}.graphml'
nx.write_graphml(G, MN_output)


### Statistics
还是有152 个nan和91个'',最后需要check一下

In [ ]:
# 
N_SPECTRA = len(CCMSIDs)
G = nx.read_graphml(TANIMOTO_GRAPHML_FILE)
N_PAIRs = G.number_of_edges() # Tanimoto score >=0.7
NPSUPERCLASSES = []
for CCMSID in CCMSIDs:
    try:
        SUPERCLASS = GNPS_INFO[CCMSID]['np_classifier_superclass']
        # print(SUPERCLASS)
        NPSUPERCLASSES.append(SUPERCLASS)
    except:pass
     
NPSUPERCLASSES_DICT = Counter(NPSUPERCLASSES)

print(f'Total number of spectra and compounds: {N_SPECTRA}, number of high similar pairs: {N_PAIRs}')
print(NPSUPERCLASSES_DICT)

In [ ]:
len(IDXtoCCMSID)
2572/2*2571

In [ ]:
# Count positive (tanimoto score >= 0.7)
POS_PAIRs = 0
for idx1 in trange(len(IDXtoCCMSID)):
    for idx2 in range(idx1):
        TANIMOTO_SCORE = TANIMOTO_MATRIX[idx1,idx2]
        if TANIMOTO_SCORE >= 0.7:
            POS_PAIRs += 1

print(f'There are {POS_PAIRs} positive pairs, accounting for {POS_PAIRs/3306306*100} total pairs.')


## Network generation

In [ ]:
# Load data
# Load gnps-library json
gnps_info = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

# Load spectral similarity matrices
N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/cosine_peak_matrix.npy' # expect for modified_cosine and neutral_loss
N_PEAK_MATRIX = np.load(N_PEAK_MATRIX_FILE)

# Load structural similarity matrices
tanimoto_matrix = np.load(f'../msdb/data/matrix/tanimoto_similarity_matrix.npy')
IDXtoCCMSID = functions_new.json_load('../msdb/data/structural_matrix/INDEXtoCCMS.json')
CCMSIDtoIDX = {value:key for key,value in IDXtoCCMSID.items() }

In [ ]:
# check if there is any spectral algorithm missing, peak percentage can directly calculated by peak matrices
group1,group2,group3,group4 = evaluation.get_groups()
ALGORITHMs = group1 + group2 + group3 + group4 # get all the spectral algorithms

MISSED_ALGORITHMs = []
for algorithm in group:
    file = f'../msdb/data/matrix/{algorithm}_similarity_matrix.npy'
    if not os.path.exists(file):
        MISSED_ALGORITHMs.append(algorithm)
INTERSECTION_ALGORITHMs = list(set(ALGORITHMs) - set(MISSED_ALGORITHMs)) # Take the intersection
MISSED_ALGORITHMs, len(ALGORITHMs), len(INTERSECTION_ALGORITHMs)

In [ ]:
# Load data
# Load gnps-library json
gnps_info = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

# Load spectral similarity matrices
N_PEAK_MATRIX_FILE = f'../msdb/data/matrix/cosine_peak_matrix.npy'  # expect for modified_cosine and neutral_loss
N_PEAK_MATRIX = np.load(N_PEAK_MATRIX_FILE)

# Load structural similarity matrices
tanimoto_matrix = np.load(f'../msdb/data/matrix/tanimoto_similarity_matrix.npy')
IDXtoCCMSID = functions_new.json_load('../msdb/data/structural_matrix/INDEXtoCCMS.json')
CCMSIDtoIDX = {value: key for key, value in IDXtoCCMSID.items()}


# Generate networks with various algorithm/spectral similarity/shared peaks
SPEC_THRESHOLD = 0.7
PEAK_THRESHOLD = 6

# group1, group2, group3, group4 = evaluation.get_groups()
# ALGORITHMs = group1 + group2 + group3 + group4 + ['peak_percentage'] # get all the spectral algorithms
ALGORITHMs = ['modified_cosine']
for ALGORITHM in tqdm(ALGORITHMs, total=len(ALGORITHMs)):
    SPEC_SIM_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy'
    SPEC_SIM_MATRIX = np.load(SPEC_SIM_MATRIX_FILE)

    topology.refMN_generate(SPEC_THRESHOLD, PEAK_THRESHOLD, SPEC_SIM_MATRIX, IDXtoCCMSID, ALGORITHM, gnps_info)

In [ ]:
# Check if the order of two index lists is the same
info1 = functions_new.json_load('../msdb/data/structural_matrix/tanimoto_INDEXtoCCMS.json')
info2 = functions_new.json_load('../msdb/data/structural_matrix/INDEXtoCCMS.json')
info1 == info2

In [ ]:
# Check if the two calculations are the same 
# They are not same, why?
ALGORITHM = 'penrose_size'
SPEC_SIM_MATRIX_FILE0 = f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix0.npy'
SPEC_SIM_MATRIX0 = np.load(SPEC_SIM_MATRIX_FILE)
SPEC_SIM_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy'
SPEC_SIM_MATRIX = np.load(SPEC_SIM_MATRIX_FILE)
SPEC_SIM_MATRIX0 == SPEC_SIM_MATRIX

In [ ]:
# Check if the matrices are calculated one-to-one by matrices
# ALGORITHM = ALGORITHMs[2]

idx1,idx2 = 333,35
CCMSID1,CCMSID2 = IDXtoCCMSID[str(idx1)],IDXtoCCMSID[str(idx2)]
SPEC1,SPECTRUM1 = functions_new.GNPS_info_format(gnps_info,CCMSID1)
SPEC2,SPECTRUM2 = functions_new.GNPS_info_format(gnps_info,CCMSID2)


for ALGORITHM in ['neutral_loss']:
    try:
        SPEC_SIM_MATRIX_FILE = f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy'
        SPEC_SIM_MATRIX = np.load(SPEC_SIM_MATRIX_FILE)
        if ALGORITHM == 'neutral_loss':
            SIMILARITY = round(peak_tools.neutral_loss(SPECTRUM1, SPECTRUM2, 0.05), 2)
            
        else:
            SIMILARITY = round(spectral_entropy.similarity(SPEC1, SPEC2, method=ALGORITHM, ms2_da=0.05), 2)
        
        if SIMILARITY !=  SPEC_SIM_MATRIX[idx1,idx2]:
            print(ALGORITHM,SIMILARITY,SPEC_SIM_MATRIX[idx1,idx2])
    except:print(ALGORITHM,SIMILARITY,SPEC_SIM_MATRIX[idx1,idx2])


### TopK

In [ ]:
# Parameters
G1,G2,G3,G4 = evaluation.get_groups()
ALGORITHMs = G1+G2+G3+G4+['peak_percentage']
ALGORITHM = [a for a in ALGORITHMs if 'modified_cosine' in a][0]
SPEC_THRESHOLD = 0.7
PEAK_THRESHOLD = 6

# Netwokr files
GRAPHML_FILE = f'../msdb/data/network/MN_{ALGORITHM}_{SPEC_THRESHOLD}_{PEAK_THRESHOLD}.graphml'

In [ ]:
# topK curating

TOPK = 5
def topK(GRAPHML_FILE,TOPK):
    
    G = nx.read_graphml(GRAPHML_FILE)
    CLUSTERs = sorted([CLUSTER for CLUSTER in nx.connected_components(G) if len(CLUSTER) > 1],key = len, reverse=True)
    for CLUSTER in CLUSTERs:
        for NODEID in CLUSTER:
            ADJ_NODEs = sorted(G[NODEID].items(),
                               key = lambda item:item[1][f'{ALGORITHM}_similarity'], reverse=True) # Sort adjacenct nodes descending
            ADJ_NODEIDs = [ADJ_NODE[0] for ADJ_NODE in ADJ_NODEs]
            N_ADJ_NODEs = len(G[NODEID]) 
            if N_ADJ_NODEs > TOPK:
                # 只保留前5条边
                # TOP5_NODEs = ADJ_NODEs[:TOPK]
                TOP5_NODEIDs = [ADJ_NODEID[0] for ADJ_NODEID in ADJ_NODEs[:TOPK]]
                NODEstoRemove = set(ADJ_NODEIDs).symmetric_difference(set(TOP5_NODEIDs))
                for NODEtoRemove in NODEstoRemove:
                    G.remove_edge(NODEID,NODEtoRemove)
                    
    DIR, FILENAME = os.path.split(GRAPHML_FILE)
    MN_OUTPUT = DIR+f'/{FILENAME.replace(".graphml","")}_top{TOPK}.graphml'
    nx.write_graphml(G, MN_OUTPUT)

In [ ]:
N20 = evaluation.n20(MN_OUTPUT)
NETWORK_ACC,CLUSTER_AVE_ACC,N_CLUSTER,N_CLUSTER_NODE = evaluation.network_accuracy(MN_OUTPUT, TANIMOTO_MATRIX_FILE, IDXtoCCMSID_FILE)
RCCC = evaluation.ratio_of_correctly_classified_component(MN_OUTPUT,GNPS_LIBRARY_FILE)
N20,NETWORK_ACC,RCCC,CLUSTER_AVE_ACC

## Topology evaluation
N20, network accuracy, correctly classified cluster

In [ ]:
# Load graphml files
TANIMOTO_MATRIX_FILE = '../msdb/data/structural_matrix/tanimoto_similarity_matrix.npy'
IDXtoCCMSID_FILE = '../msdb/data/structural_matrix/tanimoto_INDEXtoCCMS.json'
GNPS_INFO_FILE = '../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json'
GNPS_LIBRARY_FILE = '../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json'


SPEC_THRESHOLD = 0.7 # Thresholds
PEAK_THRESHOLD = 6

DIR = '../msdb/data/network/'
GRAPHML_FILEs = [DIR+f for f in os.listdir(DIR) if 'MN' and f'{SPEC_THRESHOLD}' and f'{PEAK_THRESHOLD}' in f]
gnps_info = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

# Evaluation
N20 = evaluation.n20(GRAPHML_FILEs[0])
NETWORK_ACC,CLUSTER_AVE_ACC,N_CLUSTER,N_CLUSTER_NODE = evaluation.network_accuracy(GRAPHML_FILEs[0], TANIMOTO_MATRIX_FILE, IDXtoCCMSID_FILE)
RCCC = evaluation.ratio_of_correctly_classified_component(GRAPHML_FILEs[0],GNPS_LIBRARY_FILE)

N20, NETWORK_ACC,RCCC,N_CLUSTER,N_CLUSTER_NODE

In [ ]:
# Topology_evaluation
# N20, network accuracy, correctly classified cluster
g1,g2,g3,g4 = evaluation.get_groups()
ALGORITHMs = g1+g2+g3+g4+['peak_percentage']

SPEC_THRESHOLD = 0.7
PEAK_THRESHOLD = 6
ALGOs,N20s,NETWORK_ACCs,RCCCs,N_CLUSTERs,N_CLUSTER_NODEs= [],[],[],[],[],[]
for ALGORITHM in tqdm(ALGORITHMs,total = len(ALGORITHMs)):
    try:
        GRAPHML_FILE = f'../msdb/data/network/MN_{ALGORITHM}_{SPEC_THRESHOLD}_{PEAK_THRESHOLD}.graphml'
        N20 = evaluation.n20(GRAPHML_FILE)
        NETWORK_ACC,CLUSTER_AVE_ACC,N_CLUSTER,N_CLUSTER_NODE = evaluation.network_accuracy(GRAPHML_FILE, TANIMOTO_MATRIX_FILE, IDXtoCCMSID_FILE)
        RCCC = evaluation.ratio_of_correctly_classified_component(GRAPHML_FILE,GNPS_LIBRARY_FILE)
        ALGOs.append(ALGORITHM)
        N20s.append(N20)
        NETWORK_ACCs.append(NETWORK_ACC)
        N_CLUSTERs.append(N_CLUSTER)
        N_CLUSTER_NODEs.append(N_CLUSTER_NODE)
        RCCCs.append(RCCC)
    except:
        print(ALGORITHM)   

In [ ]:
# Save as pd.DataFrame
df = pd.DataFrame({'Algorithm':ALGOs,
                   'N20':N20s,
                   'Network accuracy':NETWORK_ACCs,
                   'N_cluster':N_CLUSTERs,
                   'N_cluster_node':N_CLUSTER_NODEs
                  })
df.sort_values(by='Network accuracy', ascending=False)

### Check MS2 comparison
The performance of modified cosine is poor

In [ ]:
# Load data
# Parameters

# Netwokr files
GRAPHML_FILE = f'../msdb/data/network/MN_{ALGORITHM}_{SPEC_THRESHOLD}_{PEAK_THRESHOLD}.graphml'


In [ ]:
ALGORITHM

In [ ]:
# Check MS2 comparisons in clusters 
# Recognize clusters
G = nx.read_graphml(GRAPHML_FILE)
CLUSTERs = sorted([c for c in nx.connected_components(G) if len(c) > 1],key=len,reverse=True) # sort descending

subgraph = G.subgraph(CLUSTERs[3])
NODEIDs = list(subgraph.nodes())

CCMSID1,CCMSID2 = NODEIDs[0],NODEIDs[1]
for i in trange(len(NODEIDs)):
    CCMSID1 = NODEIDs[i]
    idx1 = int(CCMSIDtoIDX[CCMSID1])
    for j in range(i):
        CCMSID2 = NODEIDs[j]
        idx2 = int(CCMSIDtoIDX[CCMSID2])
        RESULT = functions_new.refms_compare(GNPS_INFO,ALGORITHM,CCMSID1,CCMSID2)
        if ALGORITHM == 'modified_cosine' or ALGORITHM == 'neutral_loss':
            if SPEC_SIM_MATRIX[idx1,idx2] == round(RESULT.score,2) and N_PEAK_MATRIX[idx1,idx2] == RESULT.matches:
                pass
            else: 
                print(ALGORITHM) 
                break

        else:
            if SPEC_SIM_MATRIX[idx1,idx2] == round(RESULT,2):
                pass
            else: 
                print(ALGORITHM) 
                break

            
# result,result.score,result.matches,SPEC_SIM_MATRIX[int(idx1),int(idx2)],N_PEAK_MATRIX[int(idx1),int(idx2)]

# Algorithm evaluation
TP, FP, TN, FN, ROC, FDR

### ROC curve
Add class distribution
In sklearn.metrices: TN:`C_{0,0}`, FN:`C_{1,0}`, TP: `C_{1,1}` and FP: `C_{0,1}`.

In [ ]:
# Load Data and parameters
# Parameters 
CHEM_THRESHOLD = 0.7
SPEC_THRESHOLD = 0.7 # Thresholds
PEAK_THRESHOLD = 6

ALGORITHMs = G1 + G2 + G3 + G4 + ['peak_percentage'] 
# ALGORITHMs = ['peak_percentage']

In [ ]:
ALGORITHMs

In [ ]:
# ROC curve without shared peak matrix
# Save roc_auc curve 
plt.figure(figsize=(6, 6), constrained_layout=True)
AUC_DICT = {}
for ALGORITHM in tqdm(ALGORITHMs):
    # try:
        SPEC_SIM_MATRIX = np.load(f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy')
        LABELs,SPEC_SIMILARITYs = [],[]
        TEMP_INFO = {}
        for idx1 in range(len(IDXtoCCMSID)):    
            for idx2 in range(idx1):
                CHEM_SIM = TANIMOTO_MATRIX[idx1,idx2]
                SPEC_SIM = SPEC_SIM_MATRIX[idx1,idx2]
                SPEC_SIMILARITYs.append(SPEC_SIM)
                
                if CHEM_SIM >= CHEM_THRESHOLD:
                    LABELs.append(1)
                else:
                    LABELs.append(-1)
                    
        FPRs,TPRs, THRESHOLDs = roc_curve(LABELs, SPEC_SIMILARITYs)
        AUC = auc(FPRs,TPRs)
        YOUDEN = [TPRs[i] - FPRs[i] for i in range(len(THRESHOLDs))]
        OPTIMAL_THRESHOLD = THRESHOLDs[YOUDEN.index(max(YOUDEN))] # Optimal threshold
        AUC_DICT[f'{ALGORITHM}'] = {'AUC':AUC,'OT':OPTIMAL_THRESHOLD}
        if ALGORITHM == 'modifeid_cosine':
            plt.plot(FPRs, TPRs, color="red",alpha = 0.7, lw= 1.5, label=f"ROC curve (area = {AUC:.3f})")
        if ALGORITHM == 'entropy':
            plt.plot(FPRs, TPRs, color="blue",alpha = 0.7, lw = 1.5, label=f"ROC curve (area = {AUC:.3f})")            
        else:
            plt.plot(FPRs, TPRs, color="grey",alpha = 0.7, lw = 0.8, label=f"ROC curve (area = {AUC:.3f})")
    # except:print(ALGORITHM)
        

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel("False Positive Rate", fontname='Arial', fontsize=12)
plt.ylabel("True Positive Rate", fontname='Arial', fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=12,
                labelfontfamily ='Arial', labelcolor='black', width=1, length=2)
plt.gca().set_aspect('equal', adjustable='box')

# plt.show()

# plt.title("Receiver Operating Characteristic")
# plt.plot([0, 1], [0, 1], color="navy", lw=1, linestyle="--")
# plt.savefig("../msdb/data/pic/ROC_AUC.svg", format="svg", bbox_inches="tight")

plt.savefig("../msdb/data/pic/ROC_AUC.png", dpi=300, bbox_inches="tight")

with open("../msdb/data/pic/ROC_AUC.json",'w') as f: # Save ROC_AUC json
    json.dump(AUC_DICT,f)

### FDR curve

In [ ]:
N_MS1MATCH = 0
for idx1 in trange(len(IDXtoCCMSID)):    
    for idx2 in range(idx1):
        if MS1_MATCH_MATRIX[idx1,idx2] == 1:
            N_MS1MATCH += 1 
print(N_MS1MATCH)

In [ ]:
# FDR v.s. similarity plotting
FDR_DICT = {}
for ALGORITHM in tqdm(['modified_cosine']):#tqdm(ALGORITHMs[:1]):
    try:
        SPEC_SIM_MATRIX = np.load(f'../msdb/data/matrix/{ALGORITHM}_similarity_matrix.npy')
        LABELs,SPEC_SIMILARITYs = [],[]
        TEMP_INFO = {}
        for idx1 in range(len(IDXtoCCMSID)):    
            for idx2 in range(idx1):
                CHEM_SIM = TANIMOTO_MATRIX[idx1,idx2]
                SPEC_SIM = SPEC_SIM_MATRIX[idx1,idx2]          
                
                SPEC_SIMILARITYs.append(SPEC_SIM)
                if CHEM_SIM >= CHEM_THRESHOLD:
                    LABELs.append(1)
                else:
                    LABELs.append(-1)
                    
        FPRs,TPRs, THRESHOLDs = roc_curve(LABELs, SPEC_SIMILARITYs)
        FDRs = [] # Calculate FDR
        for THRESHOLD in tqdm(THRESHOLDs,total=len(THRESHOLDs)):
            Y_PREDCTIONs = []
            for idx1 in range(len(IDXtoCCMSID)):
                for idx2 in range(idx1):
                    SPEC_SIM = SPEC_SIM_MATRIX[idx1,idx2]
                    MS1_MATCH = MS1_MATCH_MATRIX[idx1,idx2]
                    if SPEC_SIM >= THRESHOLD and MS1_MATCH == 1:
                        Y_PREDCTIONs.append(1)
                    else: 
                        Y_PREDCTIONs.append(-1)
            CONFUSION_METRIX = confusion_matrix(LABELs, Y_PREDCTIONs)  
            
            TP,FP = CONFUSION_METRIX[1,1],CONFUSION_METRIX[0,1]
            if (TP + FP) == 0:
                FDRs.append(0) 
            else:
                FDRs.append(FP/(TP+FP))

        plt.plot(THRESHOLDs,FDRs,'b*-',label = f'ROC curve of {ALGORITHM}')
    except:print(ALGORITHM)
        
# plt.plot([0,1],[0,1],'r--',label = '45° reference')
plt.legend()
plt.xlabel('Spectral similarity score')
plt.ylabel('False-discovery rate')
plt.show()

### Spec sim heatmap hist


In [ ]:
# Load spectral similarity matrices
G1,G2,G3 = evaluation.get_groups()
TALGOs = G1 + G2
TALGOs.remove('modified_cosine')

# ALGORITHMs.insert(0, 'cosine')
# ALGORITHMs.insert(0, 'modified_cosine')

In [ ]:
# 1D hist
DIR = f'../msdb/data/hqtof/SpecSimMatrix/'
ALGORITHMs = ['cosine']
for ALGO in ALGORITHMs:
    plt.figure(figsize=(6, 1.2), constrained_layout=True)
    
    SIMILARITY = [] # Hist data    
    SPEC_SIM_MATRIX = np.load(os.path.join(DIR,f'{ALGO}.npy'))
    for idx1 in range(len(SPEC_SIM_MATRIX)):
        for idx2 in range(idx1):
            SIMILARITY.append(SPEC_SIM_MATRIX[idx1,idx2])
    sns.kdeplot(
        data=SIMILARITY,
        # x=xlabel,
        lw =0.5,
        clip=(0, 1),
        legend=True,
        color="black",
        fill=True,
        bw_method= 0.2,
        warn_singular=False
        )
    plt.axis('off')
    # plt.xticks([])
    # plt.yticks([])
    plt.savefig(f'../msdb/data/pic/hist/{ALGO}_hist.png', dpi=300, bbox_inches= 'tight',pad_inches=0.01)
    plt.show()

In [ ]:
# Load spectral similarity matrices
# Param

DIR = f'../msdb/data/hqtof/' # former: f'../msdb/data/hqtof/matrix/'
ALGORITHMs = ['modified_cosine']
OTHERs = [a for a in TALGOs if a not in ALGORITHMs]
OTHERs = ['cosine']
BINs = 75

for idx1 in range(len(OTHERs)):
    ALGORITHM1 = OTHERs[idx1]
    SPEC_SIM_MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM1}.npy'))
    SIMILARITY1 = []
    for i in range(len(SPEC_SIM_MATRIX1)):
        for j in range(i):
            SIMILARITY1.append(SPEC_SIM_MATRIX1[i,j])
    
    for idx2 in range(len(ALGORITHMs)):
        ALGORITHM2 = ALGORITHMs[idx2]
        SPEC_SIM_MATRIX2 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM2}.npy'))
        SIMILARITY2 = []
        for i in range(len(SPEC_SIM_MATRIX2)):
            for j in range(i):
                SIMILARITY2.append(SPEC_SIM_MATRIX2[i,j])
                
        
        tick_locators = mticker.FixedLocator(np.arange(0, BINs + 1, BINs / 4))
        tick_labels = np.asarray([f"{a:.2f}" for a in np.arange(0, 1.01, 0.25)])
        
        fig, axe = plt.subplots(constrained_layout=True,figsize=(6, 6))
        
        hist, _, _ = np.histogram2d(
            SIMILARITY1, # X label
            SIMILARITY2, # Y label
            bins = BINs,
            range=[[0, 1], [0, 1]],
        )
        hist /= len(SIMILARITY1)
        heatmap = sns.heatmap(
            np.rot90(hist),
            vmin=0.0,
            vmax=0.001,
            cmap="viridis",
            cbar = i == 2,
            cbar_ax = cbar_ax if i == 2 else None,
            # cbar=True, # Legend 
            cbar_kws={"format": mticker.StrMethodFormatter("{x:.3%}")},
            square=True,
            xticklabels=False,
            yticklabels=False,
            norm=LogNorm(vmax=0.001),
            ax = axe
        )
        
        # # axe.plot([0, hist.shape[0]], [hist.shape[0] , 0], color='red', linewidth=1,linestyle="--")
        axe.yaxis.set_major_locator(tick_locators)
        axe.xaxis.set_major_locator(tick_locators)
        axe.set_yticklabels(tick_labels[::-1]) # Reverse
        axe.set_xticklabels(tick_labels)
        axe.set_xlabel(ALGORITHM1, fontname='Arial', fontsize=12)
        axe.set_ylabel(ALGORITHM2, fontname='Arial', fontsize=12)
        
        axe.tick_params(axis='both', which='major', labelsize=12, labelfontfamily ='Arial', width=1, length=5) # Font settings
        # axe.xaxis.set_major_formatter(mticker.PercentFormatter())
        for _, spine in heatmap.spines.items():
            spine.set_visible(True)

        
        axe.plot(
            [0, BINs], [BINs, 0], color="red", linestyle="dashed"
        )
        
        sns.despine(ax=axe) # Remove top and right border
        
    
        # 显示图形
        # plt.show()
        plt.savefig(os.path.join(DIR,f'2Dhist/{ALGORITHM2}Y_{ALGORITHM1}X_2Dhist.png'), dpi=500, bbox_inches= 'tight',pad_inches=0.5)
        # plt.close()
        

In [ ]:
os.listdir('../msdb/data/hqtof/SpecSimMatrix')

In [ ]:
# Locate specific 
DIR = '../msdb/data/hqtof/'
ALGORITHM1 = 'modified_cosine'
MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM1}.npy'))

G1,G2,G3 = evaluation.get_groups()
TAs = G1+G2+G3
TAs = ['dot_product_reverse']
for ALGORITHM2 in TAs:
    # ALGORITHM2 = 'entropy' #ALGORITHMs[0]
    MATRIX2 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM2}.npy'))
    
    mask = np.triu_indices_from(MATRIX1, k=0) # The upper triangle without diagonal
    count = np.sum(MATRIX2[mask] > MATRIX1[mask]) # 比较 MATRIX2 > MATRIX1 的位置
    
    n = MATRIX1.shape[0]
    upper_indices = np.triu_indices(n, k=0) # The upper triangle without diagonal
    total_count = len(upper_indices[0])
    
    pp = round(count/total_count,2)*100
    
    print(f'{ALGORITHM2} socore higher than {ALGORITHM1} in {pp}% spectral pairs.')

### Algorithm serious

In [ ]:
['weighted_dot_product','dot_product_reverse','dot_product','neutral_loss','modified_cosine']
['vicis_symmetric_chi_squared_3','probabilistic_symmetric_chi_squared','symmetric_chi_squared']
['squared_euclidean','euclidean']
['ms_for_id','ms_for_id_v1']
['entropy','unweighted_entropy']

### Spec Chem distribution
Violin plot

In [ ]:
G1,G2,G3 = topology.get_groups()
ALGOs = G1+G2+G3
# ALGORITHM = [a for a in ALGORITHMs if 'modified_cosine' in a][0]

In [ ]:
 # Load spec matrix
DIR = '../msdb/data/hqtof/matrix/'
ALGO = 'dice'
for ALGO in tqdm(ALGOs,total=len(ALGOs)):
    try:
        ChemSimMat = np.load('../msdb/data/hqtof/chem/ChemSim.npy')
        SpecSimMat = np.load(os.path.join(DIR,f'{ALGO}_similarity_matrix.npy'))
        
        # High sim pairs
        HSIdx = [np.where(arr >=0.7) for arr in ChemSimMat] # [(),(),...] list containing tuples
        HSSpecSim = [SpecSimMat[idx][HSIdx[idx]] for idx in range(len(HSIdx)) if HSIdx[idx][0].any()]
        # HSChemSim = [ChemSimMat[idx][HSIdx[idx]] for idx in range(len(HSIdx)) if HSIdx[idx][0].any()] 
        
        HS_values = np.concatenate(HSSpecSim)
        
        # Low sim pairs
        LSIdx = [np.where((arr >= 0.1) & (arr < 0.35)) for arr in ECHEM_MATRIX] 
        LSSpecSim = [SpecSimMat[idx][LSIdx[idx]] for idx in range(len(LSIdx)) if LSIdx[idx][0].any()]
        LSChemSim = [ChemSimMat[idx][LSIdx[idx]] for idx in range(len(LSIdx)) if LSIdx[idx][0].any()] 
        
        LS_values = np.concatenate(LSSpecSim)
        
        
        
        data_ls = pd.DataFrame({'Values': LS_values, 'Type': 'LS'})
        data_hs = pd.DataFrame({'Values': HS_values, 'Type': 'HS'})
        
        # 创建一个共享 x 轴的双 y 轴图
        fig, ax1 = plt.subplots(figsize=(3, 6))
        
        # 绘制 LS_values 的小提琴图
        sns.violinplot(x='Type', y='Values', data=data_ls, ax=ax1, color='blue', width=0.5,inner=None)
        ax1.set_title(f"{ALGO}")
        ax1.set_ylabel("LS Values", color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')
        
        # 隐藏 ax1 的边框
        for spine in ax1.spines.values():
            spine.set_visible(False)
        ax1.xaxis.set_ticks_position('none')  # 隐藏 x 轴刻度
        ax1.yaxis.set_ticks_position('none')  # 隐藏 y 轴刻度
        
        ax2 = ax1.twinx() # 创建第二个 y 轴
        
        ax2.spines['right'].set_position(('outward', -3)) # 调整 ax2 的位置，使其更靠近 ax1
        
        # 绘制 HS_values 的小提琴图
        sns.violinplot(x='Type', y='Values', data=data_hs, ax=ax2, color='red', width=0.5,inner=None)
        ax2.set_ylabel("HS Values", color='red')
        ax2.tick_params(axis='y', labelcolor='red')
        
        
        # 隐藏 ax2 的边框
        for spine in ax2.spines.values():
            spine.set_visible(False)
        
        # 调整布局
        plt.tight_layout()
        plt.savefig(f'../msdb/data/hqtof/pic_violin/{ALGO}.png', dpi=500, bbox_inches="tight")
        plt.close()
    except:print(ALGO)

In [ ]:
# View
print(FS_SPECTRA[1]['smile'])
print(FS_SPECTRA[113]['smile'])

In [ ]:
FS_SPECTRA[113]['peaks']

## Detailed inspection

In [ ]:
def clac_spec_sim(SPEC1,SPECTRUM1,SPEC2,SPECTRUM2,ALGO):
    ''''''
    cosine = CosineGreedy(tolerance=0.05)
    neutral_loss = NeutralLossesCosine(tolerance=0.05)
    modified_cosine = ModifiedCosine(tolerance=0.05)

    if ALGO == 'modified_cosine':
        score = modified_cosine.pair(SPECTRUM1, SPECTRUM2)
        SPEC_SIM = score['score'].item()
        N_PEAK = score['matches'].item()

    elif ALGO == 'cosine':
        score = cosine.pair(SPECTRUM1, SPECTRUM2)
        SPEC_SIM = score['score'].item()
        N_PEAK = score['matches'].item()

    elif ALGO == 'neutral_loss':
        score = neutral_loss.pair(SPECTRUM1, SPECTRUM2)
        SPEC_SIM = score['score'].item()
        N_PEAK = score['matches'].item()

    elif ALGO == 'peak_percentage':
        score = cosine.pair(SPECTRUM1, SPECTRUM2)
        N_PEAK = score['matches'].item()
        SPEC_SIM = N_PEAK / min(len(SPEC1), len(SPEC2))

    else:
        score = cosine.pair(SPECTRUM1, SPECTRUM2)
        N_PEAK = score['matches'].item()
        SPEC_SIM = similarity(SPEC1, SPEC2, method=ALGO, ms2_da=0.05)

    return SPEC_SIM,N_PEAK

In [ ]:
['weighted_dot_product','dot_product_reverse','dot_product','neutral_loss','modified_cosine']
['vicis_symmetric_chi_squared_3','probabilistic_symmetric_chi_squared','symmetric_chi_squared']
['squared_euclidean','euclidean']
['ms_for_id','ms_for_id_v1']
['entropy','unweighted_entropy']

In [ ]:
Hqtof_CCMSIDs = list(np.load('../msdb/data/hqtof/idlist/H_qtof_non-redundant_CCMSIDs.npy'))
# Load processed json file
GNPS_INFO = functions_new.json_load('../msdb/GNPSLIBRARY_250514/GNPS-LIBRARY-INFO.json')

modified_cosine = ModifiedCosine(tolerance = 0.05)
neutral_loss = NeutralLossesCosine(tolerance = 0.05)
cosine = CosineGreedy(tolerance=0.05)

In [ ]:
optimal_threshold = {'modified_cosine': 0.72, 'weighted_dot_product': 0.89, 'hellinger': 0.18, 'ms_for_id': 0.205, 'manhattan': 0.535,
            'absolute_value': 0.535, 'intersection': 0.535, 'lorentzian': 0.52, 'ruzicka': 0.365, 'motyka': 0.535, 'fidelity': 0.72,
            'bhattacharya_2': 0.755, 'matusita': 0.475, 'bhattacharya_1': 0.765, 'squared_chord': 0.72, 'vicis_symmetric_chi_squared_3': 0.625,
            'probabilistic_symmetric_chi_squared': 0.67, 'harmonic_mean': 0.67, 'unweighted_entropy': 0.700, 'improved_similarity': 0.515,
            'clark': 0.515, 'spectral_contrast_angle': 0.72, 'pearson_correlation': 0.845, 'dot_product': 0.72, 'inner_product': 0.1,
            'whittaker_index_of_association': 0.11, 'dot_product_reverse': 0.99, 'avg_l': 0.175, 'entropy': 0.535, 'roberts': 0.285,
            'baroni_urbani_buser': 0.735, 'neutral_loss': 0.72, 'jaccard': 0.54, 'dice': 0.52, 'peak_percentage': 0.71, 'squared_euclidean': 0.975,
            'euclidean': 0.84, 'penrose_shape': 0.84, 'chebyshev': 0.885, 'ms_for_id_v1': 0.98, 'canberra': 0.055, 'divergence': 0.03, 'wave_hedges': 0.055,
            'penrose_size': 0.23, 'mean_character': 0.985, 'symmetric_chi_squared': 0.950}

In [ ]:
DIR = '../msdb/data/hqtof'
ALGORITHM1 = 'modified_cosine'
MATRIX1 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM1}.npy'))

ALGORITHM2 = 'bhattacharya_1'
MATRIX2 = np.load(os.path.join(DIR,f'SpecSimMatrix/{ALGORITHM2}.npy'))

ChemSimMat = np.load(os.path.join(DIR,f'chem/hqtof_dice.npy'))

# Get higher
i, j = np.triu_indices_from(MATRIX1, k=1)  # The upper triangle without diagonal
mask_condition = MATRIX2[i, j] > MATRIX1[i, j]
indices = (i[mask_condition], j[mask_condition])
pair_indices = list(zip(indices[0],indices[1]))


TPs, FPs, TNs, FNs = [],[],[],[]
for i in tqdm(pair_indices,total = len(pair_indices)):
    idx1,idx2 = i
    SIM1 = MATRIX1[idx1,idx2]
    SIM2 = MATRIX2[idx1,idx2]
    diff = abs(SIM1-SIM2)
    ChemSim = ChemSimMat[idx1,idx2]
    OT = optimal_threshold[ALGORITHM2]
    if diff > 0.3:
        if SIM1 > 0 and SIM2 >= OT and ChemSim >=0.75:
            TPs.append(i)
        elif SIM1 > 0 and SIM2 >= OT and ChemSim < 0.75:
            FPs.append(i)
            
        elif SIM1 > 0 and SIM2 < OT and ChemSim >= 0.75:
            FNs.append(i)
        elif SIM1 > 0 and SIM2 < OT and ChemSim < 0.75:
            TNs.append(i)
len(TPs),len(FPs),len(TNs),len(FNs)

In [ ]:
PAIR = TPs[0]
idx1, idx2 = PAIR[0],PAIR[1]
CCMSID1,CCMSID2 = Hqtof_CCMSIDs[idx1], Hqtof_CCMSIDs[idx2]
SPEC1, SPECTRUM1 = functions_new.gnps_info_format(GNPS_INFO,CCMSID1)
SPEC2, SPECTRUM2 = functions_new.gnps_info_format(GNPS_INFO,CCMSID2)
SIM1,MP1 = clac_spec_sim(SPEC1, SPECTRUM1,SPEC2, SPECTRUM2,ALGORITHM1)
SIM2,MP2 = clac_spec_sim(SPEC1, SPECTRUM1,SPEC2, SPECTRUM2, ALGORITHM2)
PM1, PM2 = float(GNPS_INFO[CCMSID1]["PEPMASS"]),float(GNPS_INFO[CCMSID2]["PEPMASS"])
print(f'Top {CCMSID1}, precursor {PM1}')
print(f'Bottom {CCMSID2}, precursor {PM2}')
print(f"Re-calced {ALGORITHM1}:{round(SIM1,2)}",f"{ALGORITHM2}: {round(SIM2,2)}")

SMILE1 = GNPS_INFO[CCMSID1]['SMILE']
SMILE2 = GNPS_INFO[CCMSID2]['SMILE']
print(f'Matched peaks',MP,MP2,f'Chemical dice {round(cheminfo_tools.dice(SMILE1,SMILE2),2)}')
print(CCMSID1,MATRIX1[idx1,idx2], CCMSID2,MATRIX2[idx1,idx2]) # 

print(f"{SMILE1}\n{SMILE2}")

In [ ]:
# To output spectra
SPEC2

In [ ]:
# To output matched peaks and scored by scripts
spectrum_1 = sus.MsmsSpectrum(identifier='1', 
                              precursor_mz=PM1, 
                              precursor_charge=1,
                              mz=np.array(SPEC1[:, 0],dtype=np.float64),
                              intensity=np.array(SPEC1[:, 1],dtype=np.float64))
spectrum_2 = sus.MsmsSpectrum(identifier='2', 
                              precursor_mz=PM2, 
                              precursor_charge=1,
                              mz=np.array(SPEC2[:, 0],dtype=np.float64),
                              intensity=np.array(SPEC2[:, 1],dtype=np.float64)) 
MCS_RES = peaktools.modified_cosine(spectrum_1,spectrum_2,0.1)
NL_RES = peaktools.neutral_loss(spectrum_1,spectrum_2,0.1)
MCS_RES,NL_RES

In [ ]:
# MCS matched peaks and its indices
matched_indices = MCS_RES.matched_indices
matched_indices_other = MCS_RES.matched_indices_other
print(f'Shared peaks of mcs in spectrum1:{spectrum_1.mz[matched_indices]}')
print(f'Shared peaks of mcs in spectrum2:{spectrum_2.mz[matched_indices]}')

# NL shared peaks and its indices
nl_spectrum_1 = peaktools.spec_to_neutral_loss(spectrum_1)
nl_spectrum_2 = peaktools.spec_to_neutral_loss(spectrum_2)
nl_matched_indices = NL_RES.matched_indices
nl_matched_indices_other = NL_RES.matched_indices_other
PM1-PM2,nl_spectrum_1.mz[nl_matched_indices],nl_spectrum_2.mz[nl_matched_indices_other]

In [ ]:
nl_spectrum = nl_spectrum_2
np.column_stack((nl_spectrum.mz,nl_spectrum.intensity))

In [ ]:
# Matchms v
score1 = modified_cosine.pair(SPECTRUM1,SPECTRUM2)
score2 = neutral_loss.pair(SPECTRUM1,SPECTRUM2)
score1,score2

In [ ]:
SPEC1,SPEC2